In [51]:
import sys
sys.path.insert(0, "..")
from utils import linalg
from importlib import reload
import distributed
from distributed import kernel
reload(distributed.sharded_matrix)
from distributed.sharded_matrix import ShardedMatrix, ShardedSymmetricMatrix
import distributed.distributed as D
import scipy.linalg
import numpy as np
from sklearn import metrics
import boto3


In [2]:
! aws s3 ls s3://vaishaalpywrenlinalg/pywren.linalg/


                           PRE XTX(mnist_test_1)/
                           PRE XTX(mnist_test_1)_column_block_0/
                           PRE XTX(mnist_test_1)_column_block_1/
                           PRE XTX(mnist_test_1)_column_block_10/
                           PRE XTX(mnist_test_1)_column_block_11/
                           PRE XTX(mnist_test_1)_column_block_12/
                           PRE XTX(mnist_test_1)_column_block_13/
                           PRE XTX(mnist_test_1)_column_block_14/
                           PRE XTX(mnist_test_1)_column_block_2/
                           PRE XTX(mnist_test_1)_column_block_3/
                           PRE XTX(mnist_test_1)_column_block_4/
                           PRE XTX(mnist_test_1)_column_block_5/
                           PRE XTX(mnist_test_1)_column_block_6/
                           PRE XTX(mnist_test_1)_column_block_7/
                           PRE XTX(mnist_test_1)_column_block_8/
                           PRE XTX(

In [21]:
XXT_sharded = ShardedSymmetricMatrix("XXT(X_train_imagenet_fc7)", bucket="vaishaalpywrenlinalg")

In [4]:
XXT_sharded_fake = ShardedSymmetricMatrix("XXT(X_train_imagenet_fc8)", shape=XXT_sharded.shape, bucket="vaishaalpywrenlinalg")

In [28]:
XXT_sharded.blocks

[((0, 4096), (0, 4096)),
 ((0, 4096), (4096, 8192)),
 ((0, 4096), (8192, 12288)),
 ((0, 4096), (12288, 16384)),
 ((0, 4096), (16384, 20480)),
 ((0, 4096), (20480, 24576)),
 ((0, 4096), (24576, 28672)),
 ((0, 4096), (28672, 32768)),
 ((0, 4096), (32768, 36864)),
 ((0, 4096), (36864, 40960)),
 ((0, 4096), (40960, 45056)),
 ((0, 4096), (45056, 49152)),
 ((0, 4096), (49152, 53248)),
 ((0, 4096), (53248, 57344)),
 ((0, 4096), (57344, 61440)),
 ((0, 4096), (61440, 65536)),
 ((0, 4096), (65536, 69632)),
 ((0, 4096), (69632, 73728)),
 ((0, 4096), (73728, 77824)),
 ((0, 4096), (77824, 81920)),
 ((0, 4096), (81920, 86016)),
 ((0, 4096), (86016, 90112)),
 ((0, 4096), (90112, 94208)),
 ((0, 4096), (94208, 98304)),
 ((0, 4096), (98304, 102400)),
 ((0, 4096), (102400, 106496)),
 ((0, 4096), (106496, 110592)),
 ((0, 4096), (110592, 114688)),
 ((0, 4096), (114688, 118784)),
 ((0, 4096), (118784, 122880)),
 ((0, 4096), (122880, 126976)),
 ((0, 4096), (126976, 131072)),
 ((0, 4096), (131072, 135168)),
 

In [22]:
%time blocks_exist = set(XXT_sharded.blocks_exist)

CPU times: user 13.3 s, sys: 320 ms, total: 13.6 s
Wall time: 19.9 s


In [23]:
%time blocks = set(XXT_sharded.blocks)

CPU times: user 196 ms, sys: 8 ms, total: 204 ms
Wall time: 202 ms


In [50]:
tuple(sorted(list(blocks_exist)[3])) in blocks

True

In [39]:
list(blocks)[0][::-1] in blocks_exist

True

In [ ]:
X_train_sharded = ShardedMatrix("X_train_3_3_pool_12_12_patch_overlap_stride_1_normalized", bucket="vaishaalpywrenlinalg")

In [16]:
X_test_sharded = ShardedMatrix("X_test_3_3_pool_12_12_patch_overlap_stride_1_normalized", bucket="vaishaalpywrenlinalg")

In [12]:
%time X_train = D.fast_kernel_column_blocks_get(X_train_sharded, \
                                                col_blocks=X_train_sharded._block_idxs(1), \
                                                row_blocks=X_train_sharded._block_idxs(0), \
                                                mmap_loc="/dev/shm/X_train_coates_ng", \
                                                dtype="float64")

CPU times: user 8 ms, sys: 952 ms, total: 960 ms
Wall time: 2min 46s


In [26]:
%time y_train = np.load("/home/ubuntu/imagenet-data/scrambled_train_labels.npy")
%time y_test = np.load("/home/ubuntu/imagenet-data/scrambled_test_labels.npy")

CPU times: user 4 ms, sys: 8 ms, total: 12 ms
Wall time: 10 ms
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.57 ms


In [18]:
%time X_test = D.fast_kernel_column_blocks_get(X_test_sharded, \
                                                col_blocks=X_test_sharded._block_idxs(1), \
                                                row_blocks=X_test_sharded._block_idxs(0), \
                                                mmap_loc="/dev/shm/X_test_coates_ng", \
                                                dtype="float64")

CPU times: user 52 ms, sys: 144 ms, total: 196 ms
Wall time: 7.7 s


In [20]:
%time XTX = X_train.T.dot(X_train)

CPU times: user 4h 47min 52s, sys: 5h 12min 33s, total: 10h 25s
Wall time: 9min 33s


In [28]:
y_enc = np.eye(1000)[y_train.astype('int')]

In [41]:
%time XTy = X_train.T.dot(y_enc)

CPU times: user 33min 47s, sys: 27min 36s, total: 1h 1min 24s
Wall time: 59.8 s


In [31]:
def exact_solve(XTX, Xty, reg):
    idxs = np.diag_indices(XTX.shape[0])
    XTX[idxs] += reg
    try:
        model = scipy.linalg.solve(XTX, Xty)
        XTX[idxs] -= reg
    except:
        XTX[idxs] -= reg
    return model

In [52]:
model = exact_solve(XTX, XTy, reg=1000)

In [53]:
y_hat = X_train.dot(model)

In [54]:
metrics.accuracy_score(np.argmax(y_hat, axis=1), y_train)

0.18423359327862801

In [48]:
model.shape

(18432, 1000)

In [55]:
y_test_hat = X_test.dot(model)

In [56]:
metrics.accuracy_score(np.argmax(y_test_hat, axis=1), y_test)

0.1421

In [60]:
0.5 * np.linalg.norm(y_hat  - y_enc)**2

627016.66737260961

In [165]:
def list_all_keys(bucket, prefix):
    client = boto3.client('s3')
    objects = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter=prefix)
    keys = list(map(lambda x: x['Key'], objects['Contents']))
    truncated = objects['IsTruncated']
    next_marker = objects['NextMarker']
    while (truncated):
        objects = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter=prefix, Marker=next_marker)
        truncated = objects['IsTruncated']
        next_marker = objects.get('NextMarker')
        keys += list(map(lambda x: x['Key'], objects['Contents']))
    return list(filter(lambda x: x != None, keys))

def block_key_to_block(key):
    try:
        block_key = key.split("/")[-1]
        blocks_split = block_key.split("_")
        b0_start = int(blocks_split[0])
        b0_end = int(blocks_split[1])
        b1_start = int(blocks_split[3])
        b1_end = int(blocks_split[4])
        return ((b0_start, b0_end), (b1_start, b1_end))
    except:
        return None
        

def exist_blocks(X_sharded):
    prefix = X_sharded.prefix + X_sharded.key
    all_keys = list_all_keys(X_sharded.bucket, prefix)
    return list(map(block_key_to_block, all_keys))
    

In [160]:
%time blocks = exist_blocks(XXT_sharded)

CPU times: user 13.6 s, sys: 232 ms, total: 13.8 s
Wall time: 19.4 s


In [162]:
set(blocks)

{'',
 ((618496, 622592), (221184, 225280)),
 ((815104, 819200), (528384, 532480)),
 ((917504, 921600), (90112, 94208)),
 ((946176, 950272), (397312, 401408)),
 ((753664, 757760), (663552, 667648)),
 ((1269760, 1273856), (118784, 122880)),
 ((225280, 229376), (163840, 167936)),
 ((237568, 241664), (200704, 204800)),
 ((610304, 614400), (233472, 237568)),
 ((790528, 794624), (774144, 778240)),
 ((917504, 921600), (782336, 786432)),
 ((688128, 692224), (282624, 286720)),
 ((724992, 729088), (655360, 659456)),
 ((913408, 917504), (774144, 778240)),
 ((995328, 999424), (708608, 712704)),
 ((999424, 1003520), (368640, 372736)),
 ((1056768, 1060864), (53248, 57344)),
 ((745472, 749568), (417792, 421888)),
 ((49152, 53248), (45056, 49152)),
 ((843776, 847872), (643072, 647168)),
 ((888832, 892928), (53248, 57344)),
 ((1105920, 1110016), (471040, 475136)),
 ((122880, 126976), (32768, 36864)),
 ((1228800, 1232896), (565248, 569344)),
 ((1269760, 1273856), (1077248, 1081344)),
 ((253952, 258048),

In [146]:
prefix = XXT_sharded.prefix + XXT_sharded.key

In [153]:
all_keys

['pywren.linalg/XXT(X_train_imagenet_fc7)/0_4096_4096_0_4096_4096_0',
 'pywren.linalg/XXT(X_train_imagenet_fc7)/1003520_1007616_4096_0_4096_4096_0',
 'pywren.linalg/XXT(X_train_imagenet_fc7)/1003520_1007616_4096_1003520_1007616_4096_0',
 'pywren.linalg/XXT(X_train_imagenet_fc7)/1003520_1007616_4096_102400_106496_4096_0',
 'pywren.linalg/XXT(X_train_imagenet_fc7)/1003520_1007616_4096_106496_110592_4096_0',
 'pywren.linalg/XXT(X_train_imagenet_fc7)/1003520_1007616_4096_110592_114688_4096_0',
 'pywren.linalg/XXT(X_train_imagenet_fc7)/1003520_1007616_4096_114688_118784_4096_0',
 'pywren.linalg/XXT(X_train_imagenet_fc7)/1003520_1007616_4096_118784_122880_4096_0',
 'pywren.linalg/XXT(X_train_imagenet_fc7)/1003520_1007616_4096_122880_126976_4096_0',
 'pywren.linalg/XXT(X_train_imagenet_fc7)/1003520_1007616_4096_12288_16384_4096_0',
 'pywren.linalg/XXT(X_train_imagenet_fc7)/1003520_1007616_4096_126976_131072_4096_0',
 'pywren.linalg/XXT(X_train_imagenet_fc7)/1003520_1007616_4096_131072_135168_

In [156]:
%time all_keys = list_all_keys("vaishaalpywrenlinalg", prefix)

CPU times: user 12.5 s, sys: 184 ms, total: 12.7 s
Wall time: 18.7 s


In [76]:
sample_key = list(map(lambda x: x['Key'], objects['Contents']))[0]

In [140]:
objects['NextMarker']

'pywren.linalg/XXT(X_train_imagenet_fc7)/1019904_1024000_4096_110592_114688_4096_0'

In [85]:
block_key = os.path.split(os.path.split(sample_key)[1])[-1]

In [89]:
int(block_key.split("_")[0])

0

In [90]:

int(block_key.split("_")[1])

4096

In [92]:
block_key

'0_4096_4096_0_18432_18432_0'

In [110]:

def s3_key
block_key = os.path.split(os.path.split(sample_key)[1])[-1]

['0', '4096', '4096', '0', '18432', '18432', '0']

In [113]:
def block_key_to_block(key):
    block_key = key.split("/")[-1]
    blocks_split = block_key.split("_")
    b0_start = int(blocks_split[0])
    b0_end = int(blocks_split[1])
    b1_start = int(blocks_split[3])
    b1_end = int(blocks_split[4])
    return ((b0_start, b0_end), (b1_start, b1_end))

In [114]:
block_key_to_block(sample_key)

((0, 4096), (0, 18432))

In [102]:
X_train_sharded.blocks

[((0, 4096), (0, 18432)),
 ((4096, 8192), (0, 18432)),
 ((8192, 12288), (0, 18432)),
 ((12288, 16384), (0, 18432)),
 ((16384, 20480), (0, 18432)),
 ((20480, 24576), (0, 18432)),
 ((24576, 28672), (0, 18432)),
 ((28672, 32768), (0, 18432)),
 ((32768, 36864), (0, 18432)),
 ((36864, 40960), (0, 18432)),
 ((40960, 45056), (0, 18432)),
 ((45056, 49152), (0, 18432)),
 ((49152, 53248), (0, 18432)),
 ((53248, 57344), (0, 18432)),
 ((57344, 61440), (0, 18432)),
 ((61440, 65536), (0, 18432)),
 ((65536, 69632), (0, 18432)),
 ((69632, 73728), (0, 18432)),
 ((73728, 77824), (0, 18432)),
 ((77824, 81920), (0, 18432)),
 ((81920, 86016), (0, 18432)),
 ((86016, 90112), (0, 18432)),
 ((90112, 94208), (0, 18432)),
 ((94208, 98304), (0, 18432)),
 ((98304, 102400), (0, 18432)),
 ((102400, 106496), (0, 18432)),
 ((106496, 110592), (0, 18432)),
 ((110592, 114688), (0, 18432)),
 ((114688, 118784), (0, 18432)),
 ((118784, 122880), (0, 18432)),
 ((122880, 126976), (0, 18432)),
 ((126976, 131072), (0, 18432)),
 

In [52]:
ia

NameError: name 'ia' is not defined

In [53]:
XXT_sharded.prefix

'pywren.linalg/'

In [54]:
XXT_sharded.key

'XXT(X_train_imagenet_fc7)'